In [6]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))
from wofscast.data_generator import (add_local_solar_time, 
                                     to_static_vars, 
                                     dataset_to_input,
                                     load_chunk,
                                     load_and_concatenate, 
                                     ZarrDataGenerator,
                                     WoFSDataProcessor, 
                                     WRFZarrFileProcessor,
                                     open_zarr, check_for_nans
                                    )
from wofscast import data_utils
from wofscast.wofscast_task_config import DBZ_TASK_CONFIG, WOFS_TASK_CONFIG, DBZ_TASK_CONFIG_1HR
from os.path import join
import dataclasses

import xarray as xr
from dask.distributed import Client
import zarr

In [2]:
%%time 
# Usage
base_path = '/work2/wofs_zarr/'
years = ['2019']#, '2020']
resolution_minutes = 10

# Specify the restrictions for testing
restricted_dates = ['20190526']
restricted_times =['1900']#, '2000', '2100', '2200', '2300', '0000', '0100', '0200', '0300']
restricted_members = ['ENS_MEM_15']#, 'ENS_MEM_16']#, 'ENS_MEM_10', 'ENS_MEM_11']

processor = WRFZarrFileProcessor(base_path, years, 
                             resolution_minutes, 
                             restricted_dates, 
                             restricted_times, restricted_members)

paths = processor.run()

# All dates, one init time, one member ; 

CPU times: user 1.89 ms, sys: 518 µs, total: 2.41 ms
Wall time: 2.58 ms


In [3]:
dataset = load_and_concatenate(paths[0], concat_dim='Time')
preprocessor = WoFSdataProcessor()
dataset = preprocessor(dataset)

In [9]:
# Save to disk. 
# Configure the Blosc compressor
compressor = zarr.Blosc(cname='zstd', clevel=7, shuffle=zarr.Blosc.BITSHUFFLE)

# Save the dataset to a Zarr file
# print(f'Saving to {out_path}...')
out_path = '/work2/wofs_zarr/ai_datasets/2019/test.zarr'
dataset.to_zarr(out_path, mode='w', 
                        encoding={var: {'compressor': compressor} for var in dataset.variables})